In [102]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as T
from collections import namedtuple
import math
import pandas as pd
import numpy as np
import random

MacroFactor = ['gdpgr','pconsump','cpi','Unemploy','m3tb','tb10y','aa10y','gpdinv']
VARModel = 'C:/dsge/py/input/var.csv'
Mapping = 'C:/dsge/py/input/mapping.csv'
Mortality = 'C:/dsge/py/input/mortality.csv'
Census = 'C:/dsge/py/input/census.csv'
RecFun = [77.96,66.31,0.34,-1.23,-25.34,-51.33,-3.4,-9.86,-8.45,-32.19,4.26,0.12,-17.02]
histMF = 'C:/dsge/py/input/histMF.csv' 
histAR = 'C:/dsge/py/input/histAR.csv'
cholMF = 'C:/dsge/py/input/var1chol.csv'
cholNormal = 'C:/dsge/py/input/normalcholf.csv'
cholRecession = 'C:/dsge/py/input/recessioncholf.csv'
termmix = 'C:/dsge/py/input/termmix.csv'
migration = 'C:/dsge/py/input/migration.csv'

random.seed(6)

In [2]:
def recession(fun,vals):
	vals.insert(0,1)
	prob = 1/(1+math.exp(-sum([x*y for x,y in zip(fun, vals)])))
	if prob > 0.5:
		return 1
	else:
		return 0

In [3]:
#y = [6.358,0.617,0.921,0.454,6.357,0.680,0.931,0.463,6.349,0.732,0.940,0.471]
#recession(RecFun,y)

In [4]:
def sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,dyn,n,stochastic=True):
	var = np.array(var)
	fundmap = np.array(fundmap)
	histMF = np.array(histMF)
	histAR = np.array(histAR)
	cholMF = np.array(cholMF)
	cholNormal = np.array(cholNormal)
	cholRecession = np.array(cholRecession)
	if stochastic==True and dyn>=n:
		sd = var[:,[8]]
		var = var[:,[range(0,9)]]
		hmf = histMF
		hmf3 = np.insert(hmf,0,hmf[hmf.shape[0]-2,:],0)
		recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
		rece = recession(RecFun,recessionVal)
		hmf = hmf[hmf.shape[0]-1,:]
		thmf = np.append(hmf,rece)
		sdtnormal = np.sqrt(fundmap[:,[27]])
		sdtrecession = np.sqrt(fundmap[:,[28]])
		sdinormal = np.sqrt(fundmap[:,[29]])
		sdirecession = fundmap[:,[30]]
		corrnormal = fundmap[:,[31]]
		corrrecession = fundmap[:,[32]]
		fundmap = fundmap[:,[range(0,27)]]
		har = histAR
		har3 = np.insert(har,0,har[har.shape[0]-2,:],0)
		har = har[har.shape[0]-1,:]
		thmf = np.append(thmf,har)
		hmf=np.append(hmf,1)
		for i in range(0,n):
			hmf = var.dot(hmf.T).T
			hmf = hmf + np.multiply(sd.T,np.dot(cholMF,np.array(np.random.normal(0,1,8)).T))
			hmf3[0] = hmf3[1]
			hmf3[1] = hmf3[2]
			hmf3[2] = hmf
			recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
			rece = recession(RecFun,recessionVal)
			for j in range(0,har.shape[0]):
				hartmp = hmf3.flatten('F')
				hartmp = np.insert(hartmp,0,[1,har3[1,j],har3[2,j]])
				har[j] = fundmap[j].dot(hartmp.T).T
			if rece==0:
				rnds = np.multiply(sdinormal.T, np.dot(cholNormal,np.array(np.random.normal(0,1,33)).T))
				rnds = np.multiply((np.multiply(corrnormal.T,har) + np.multiply(np.sqrt(1-np.multiply(corrnormal.T,corrnormal.T)),rnds)),sdinormal.T)
				rnds = np.divide(rnds,np.sqrt(np.multiply(np.multiply(corrnormal.T,corrnormal.T),np.multiply(sdtnormal.T,sdtnormal.T))+np.multiply((1-np.multiply(corrnormal.T,corrnormal.T)),np.multiply(sdinormal.T,sdinormal.T))))
				rnds[0,5]=0
				rnds[0,8]=0
				rnds[0,10]=0
				rnds[0,30]=0
				rnds[0,31]=0
				har = har + rnds.flatten('F')
			else:
				rnds = np.multiply(sdirecession.T, np.dot(cholRecession,np.array(np.random.normal(0,1,33)).T))
				rnds = np.multiply((np.multiply(corrrecession.T,har) + np.multiply(np.sqrt(1-np.multiply(corrrecession.T,corrrecession.T)),rnds)),sdirecession.T)
				rnds = np.divide(rnds,np.sqrt(np.multiply(np.multiply(corrrecession.T,corrrecession.T),np.multiply(sdtrecession.T,sdtrecession.T))+np.multiply((1-np.multiply(corrrecession.T,corrrecession.T)),np.multiply(sdirecession.T,sdirecession.T))))
				rnds[0,5]=0
				rnds[0,8]=0
				rnds[0,10]=0
				rnds[0,30]=0
				rnds[0,31]=0
				har = har + rnds.flatten('F')
			for ix in range(0,8):
				har[ix]=max(0.1,har[ix])
			for ix in [10,12,14,16]:
				har[ix]=max(0,har[ix])
			for ix in range(0,har.shape[0]):
				har[ix]=max(-99,har[ix])
			har3[0] = har3[1]
			har3[1] = har3[2]
			har3[2] = har
			thmf = np.vstack([thmf, np.append(np.append(hmf,rece),har.flatten())])
			hmf=np.append(hmf,1)
		return thmf
	elif stochastic==True and dyn>0 and dyn<n:
		sd = var[:,[8]]
		var = var[:,[range(0,9)]]
		hmf = histMF
		hmf3 = np.insert(hmf,0,hmf[hmf.shape[0]-2,:],0)
		recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
		rece = recession(RecFun,recessionVal)
		hmf = hmf[hmf.shape[0]-1,:]
		thmf = np.append(hmf,rece)
		sdtnormal = fundmap[:,[27]]
		sdtrecession = fundmap[:,[28]]
		sdinormal = fundmap[:,[29]]
		sdirecession = fundmap[:,[30]]
		corrnormal = fundmap[:,[31]]
		corrrecession = fundmap[:,[32]]
		fundmap = fundmap[:,[range(0,27)]]
		har = histAR
		har3 = np.insert(har,0,har[har.shape[0]-2,:],0)
		har = har[har.shape[0]-1,:]
		thmf = np.append(thmf,har)
		hmf=np.append(hmf,1)
		for i in range(0,dyn):
			hmf = var.dot(hmf.T).T
			hmf = hmf + np.multiply(sd.T,np.dot(cholMF,np.array(np.random.normal(0,1,8)).T))
			hmf3[0] = hmf3[1]
			hmf3[1] = hmf3[2]
			hmf3[2] = hmf
			recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
			rece = recession(RecFun,recessionVal)
			for j in range(0,har.shape[0]):
				hartmp = hmf3.flatten('F')
				hartmp = np.insert(hartmp,0,[1,har3[1,j],har3[2,j]])
				har[j] = fundmap[j].dot(hartmp.T).T
			if rece==0:
				rnds = np.multiply(sdinormal.T, np.dot(cholNormal,np.array(np.random.normal(0,1,33)).T))
				rnds = np.multiply((np.multiply(corrnormal.T,har) + np.multiply(np.sqrt(1-np.multiply(corrnormal.T,corrnormal.T)),rnds)),sdinormal.T)
				rnds = np.divide(rnds,np.sqrt(np.multiply(np.multiply(corrnormal.T,corrnormal.T),np.multiply(sdtnormal.T,sdtnormal.T))+np.multiply((1-np.multiply(corrnormal.T,corrnormal.T)),np.multiply(sdinormal.T,sdinormal.T))))
				rnds[0,5]=0
				rnds[0,8]=0
				rnds[0,10]=0
				rnds[0,30]=0
				rnds[0,31]=0
				har = har + rnds.flatten('F')
			else:
				rnds = np.multiply(sdirecession.T, np.dot(cholRecession,np.array(np.random.normal(0,1,33)).T))
				rnds = np.multiply((np.multiply(corrrecession.T,har) + np.multiply(np.sqrt(1-np.multiply(corrrecession.T,corrrecession.T)),rnds)),sdirecession.T)
				rnds = np.divide(rnds,np.sqrt(np.multiply(np.multiply(corrrecession.T,corrrecession.T),np.multiply(sdtrecession.T,sdtrecession.T))+np.multiply((1-np.multiply(corrrecession.T,corrrecession.T)),np.multiply(sdirecession.T,sdirecession.T))))
				rnds[0,5]=0
				rnds[0,8]=0
				rnds[0,10]=0
				rnds[0,30]=0
				rnds[0,31]=0
				har = har + rnds.flatten('F')
			for ix in range(0,8):
				har[ix]=max(0.1,har[ix])
			for ix in [10,12,14,16]:
				har[ix]=max(0,har[ix])
			for ix in range(0,har.shape[0]):
				har[ix]=max(-99,har[ix])
			har3[0] = har3[1]
			har3[1] = har3[2]
			har3[2] = har
			thmf = np.vstack([thmf, np.append(np.append(hmf,rece),har.flatten())])
			hmf=np.append(hmf,1)
		for i in range(dyn,n):
			hmf = var.dot(hmf.T).T
			hmf3[0] = hmf3[1]
			hmf3[1] = hmf3[2]
			hmf3[2] = hmf
			recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
			rece = recession(RecFun,recessionVal)
			for j in range(0,har.shape[0]):
				hartmp = hmf3.flatten('F')
				hartmp = np.insert(hartmp,0,[1,har3[1,j],har3[2,j]])
				har[j] = fundmap[j].dot(hartmp.T).T
			for ix in range(0,8):
				har[ix]=max(0.1,har[ix])
			for ix in [10,12,14,16]:
				har[ix]=max(0,har[ix])
			for ix in range(0,har.shape[0]):
				har[ix]=max(-99,har[ix])
			har3[0] = har3[1]
			har3[1] = har3[2]
			har3[2] = har
			thmf = np.vstack([thmf, np.append(np.append(hmf,rece),har.flatten())])
			hmf=np.append(hmf,1)
		return thmf
	else:
		var = var[:,[range(0,9)]]
		hmf = histMF
		hmf3 = np.insert(hmf,0,hmf[hmf.shape[0]-2,:],0)
		recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
		rece = recession(RecFun,recessionVal)
		hmf = hmf[hmf.shape[0]-1,:]
		thmf = np.append(hmf,rece)
		sdtnormal = fundmap[:,[27]]
		sdtrecession = fundmap[:,[28]]
		sdinormal = fundmap[:,[29]]
		sdirecession = fundmap[:,[30]]
		corrnormal = fundmap[:,[31]]
		corrrecession = fundmap[:,[32]]
		fundmap = fundmap[:,[range(0,27)]]
		har = histAR
		har3 = np.insert(har,0,har[har.shape[0]-2,:],0)
		har = har[har.shape[0]-1,:]
		thmf = np.append(thmf,har)
		hmf=np.append(hmf,1)
		for i in range(0,n):
			hmf = var.dot(hmf.T).T
			hmf3[0] = hmf3[1]
			hmf3[1] = hmf3[2]
			hmf3[2] = hmf
			recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
			rece = recession(RecFun,recessionVal)
			for j in range(0,har.shape[0]):
				hartmp = hmf3.flatten('F')
				hartmp = np.insert(hartmp,0,[1,har3[1,j],har3[2,j]])
				har[j] = fundmap[j].dot(hartmp.T).T
			for ix in range(0,8):
				har[ix]=max(0.1,har[ix])
			for ix in [10,12,14,16]:
				har[ix]=max(0,har[ix])
			for ix in range(0,har.shape[0]):
				har[ix]=max(-99,har[ix])
			har3[0] = har3[1]
			har3[1] = har3[2]
			har3[2] = har
			thmf = np.vstack([thmf, np.append(np.append(hmf,rece),har.flatten())])
			hmf=np.append(hmf,1)
		return thmf

var = pd.read_csv(VARModel)
hmf = pd.read_csv(histMF)
var1chol = pd.read_csv(cholMF)
fundmap = pd.read_csv(Mapping)
har = pd.read_csv(histAR)
normalChol = pd.read_csv(cholNormal)
recessionChol = pd.read_csv(cholRecession)
thmf=sampleMF(var,fundmap,hmf,har,var1chol,normalChol,recessionChol,5,30,stochastic=True)
np.corrcoef(thmf[:,[18,20,22,24]],rowvar=False)

C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:110: RuntimeWarning: invalid value encountered in true_divide
C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in true_divide


array([[1.        , 0.90988824, 0.89295381, 0.88594982],
       [0.90988824, 1.        , 0.98948079, 0.98938919],
       [0.89295381, 0.98948079, 1.        , 0.99781682],
       [0.88594982, 0.98938919, 0.99781682, 1.        ]])

In [5]:
check = sampleMF(var,fundmap,hmf,har,var1chol,normalChol,recessionChol,60,60,stochastic=False)

In [6]:
check[60]

array([ 0.47696483,  0.97956234,  0.4350555 ,  6.07512079,  0.83325078,
        3.09886149,  1.30450305,  0.66461587,  0.        ,  1.1298329 ,
        1.47254382,  1.74174492,  2.33268568,  2.76070693,  3.09886149,
        3.64268116,  3.74922226,  0.4350555 ,  0.41996198,  0.        ,
        0.44124747,  0.02253606,  0.99874293,  0.0614174 ,  1.91552878,
        0.17948035,  2.05911226,  1.75738716,  1.37735179,  2.40865312,
        1.15172477,  2.80911583,  3.57601297,  1.55743613,  1.2837999 ,
        1.52615764,  2.88864823, -0.79530345,  0.71764074,  6.07512079,
        0.47696483,  1.85657105])

In [7]:
def sampleMF2(thmf,var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,dyn,n):
	var = np.array(var)
	fundmap = np.array(fundmap)
	histMF = np.array(histMF)
	histAR = np.array(histAR)
	cholMF = np.array(cholMF)
	cholNormal = np.array(cholNormal)
	cholRecession = np.array(cholRecession)

	sd = var[:,[8]]
	var = var[:,[range(0,9)]]
	hmf = histMF
	hmf3 = np.insert(hmf,0,hmf[hmf.shape[0]-2,:],0)
	recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
	rece = recession(RecFun,recessionVal)
	hmf = hmf[hmf.shape[0]-1,:]
	sdtnormal = fundmap[:,[27]]
	sdtrecession = fundmap[:,[28]]
	sdinormal = fundmap[:,[29]]
	sdirecession = fundmap[:,[30]]
	corrnormal = fundmap[:,[31]]
	corrrecession = fundmap[:,[32]]
	fundmap = fundmap[:,[range(0,27)]]
	har = histAR
	har3 = np.insert(har,0,har[har.shape[0]-2,:],0)
	har = har[har.shape[0]-1,:]
	hmf=np.append(hmf,1)

	thmf = thmf[range(0,dyn+1),:]
	har = thmf[dyn,range(9,42)]
	hmf = thmf[dyn,range(0,8)]

	if dyn==1:
		har3[0]=har3[1]
		har3[1]=har3[2]
		har3[2]=har
		hmf3[0] = hmf3[1]
		hmf3[1] = hmf3[2]
		hmf3[2] = hmf
	elif dyn==2:
		har3[0]=har3[2]
		har3[1]=thmf[dyn-1,range(9,42)]
		har3[2]=har
		hmf3[0] = hmf3[2]
		hmf3[1] = thmf[dyn-1,range(0,8)]
		hmf3[2] = hmf
	elif dyn>2:
		har3[0]=thmf[dyn-2,range(9,42)]
		har3[1]=thmf[dyn-1,range(9,42)]
		har3[2]=har
		hmf3[0] = thmf[dyn-2,range(0,8)]
		hmf3[1] = thmf[dyn-1,range(0,8)]
		hmf3[2] = hmf
		
	hmf=np.append(hmf,1)
	
	for i in range(dyn,n):
		hmf = var.dot(hmf.T).T
		hmf3[0] = hmf3[1]
		hmf3[1] = hmf3[2]
		hmf3[2] = hmf
		recessionVal = [hmf3[2,3],hmf3[2,7],hmf3[2,1],hmf3[2,0],hmf3[1,3],hmf3[1,7],hmf3[1,1],hmf3[1,0],hmf3[0,3],hmf3[0,7],hmf3[0,1],hmf3[0,0]]
		rece = recession(RecFun,recessionVal)
		for j in range(0,har.shape[0]):
			hartmp = hmf3.flatten('F')
			hartmp = np.insert(hartmp,0,[1,har3[1,j],har3[2,j]])
			har[j] = fundmap[j].dot(hartmp.T).T
		for ix in range(0,8):
			har[ix]=max(0.1,har[ix])
		for ix in [10,12,14,16]:
			har[ix]=max(0,har[ix])
		for ix in range(0,har.shape[0]):
			har[ix]=max(-99,har[ix])
		har3[0] = har3[1]
		har3[1] = har3[2]
		har3[2] = har
		thmf = np.vstack([thmf, np.append(np.append(hmf,rece),har.flatten())])
		hmf=np.append(hmf,1)
	return thmf

sampleMF2(thmf, var,fundmap,hmf,har,var1chol,normalChol,recessionChol,5,30)[6]
#sampleMF2(check, var,fundmap,hmf,har,var1chol,normalChol,recessionChol,5,30)[30]

array([-8.62783376e-02,  1.53692446e+00,  6.83052630e-01,  4.57697730e+00,
        9.40012108e-01,  3.56132983e+00,  1.67080709e-02, -3.88898418e+00,
        1.00000000e+00,  1.49058368e+00,  1.63922956e+00,  1.92885294e+00,
        2.74750974e+00,  3.23948178e+00,  3.56132983e+00,  3.63132472e+00,
        3.96784860e+00,  6.83052630e-01, -5.35088119e-01,  0.00000000e+00,
       -1.21390345e+00,  2.09251480e-02, -1.24408110e+00,  0.00000000e+00,
       -7.50776949e-01,  0.00000000e+00,  2.25631870e+00,  2.23779943e+01,
        1.84072282e+00,  8.39598860e+00,  1.71309315e+00,  1.41301290e+01,
        2.86673933e+00,  4.83377808e+00,  5.54340306e-01,  1.68180850e+01,
        4.01955617e+00,  3.40222502e+00,  1.20926099e+00,  4.57697730e+00,
       -8.62783376e-02,  2.86172053e+01])

In [8]:
from scipy.interpolate import interp1d
x = np.array([1,2,3,5,7,10,20])
y = np.array([0.5,1.1,1.2,1.9,2.15,2.5,3])
f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')

In [9]:
#xnew = np.linspace(1, 20, num=41, endpoint=True)
#import matplotlib.pyplot as plt
#plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
#plt.legend(['data', 'linear', 'cubic'], loc='best')
#plt.show()

In [10]:
def curvefitting(value,term=np.array([0.25,1,2,3,5,7,10,20,30]),inter="linear",extro="cfr",target=0.04):
	if value.shape[0] != term.shape[0]:
		return -1
	else:
		startT = term[0]
		endT = term[term.shape[0]-1]
		secondEndT = term[term.shape[0]-2]
		xnew = np.linspace(startT,endT,num=(endT-startT)*4+1,endpoint=True)
		if inter=="linear":
			f = interp1d(term, value)
		else:
			f = interp1d(term, value, kind='cubic')
		ynew = f(xnew)
		if endT < 100:
			xnew2 = np.linspace(endT+0.25,100,num=(100-endT)*4,endpoint=True)
			ynew2=[]
			if extro=="cfr":
				cfr = (((1+value[value.shape[0]-1]) ** endT)/((1+value[value.shape[0]-2]) ** secondEndT)) ** (1/(endT-secondEndT))-1
				cumfac = (1+value[value.shape[0]-1]) ** endT
				for ix in xnew2:
					cumfac = cumfac*(1+cfr)**(1/4)
					ynew2.append(cumfac**(1/ix)-1)
			else:
				p = np.poly1d(np.polyfit(np.array([secondEndT,endT,100]),np.array([value[value.shape[0]-2],value[value.shape[0]-1],target]),deg=2))
				ynew2 = p(xnew2)
		x = np.append(xnew,xnew2)
		y = np.append(ynew,ynew2)
		return x,y

xi,yi = curvefitting(np.array([0.25,0.3,0.5,1.1,1.2,1.9,2.15,2.5,3])/100)


C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  
C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  from ipykernel import kernelapp as app


In [11]:
#import matplotlib.pyplot as plt
#term = np.array([0.25,1,2,3,5,7,10,20,30])
#value = np.array([0.25,0.3,0.5,1.1,1.2,1.9,2.15,2.5,3])/100
#plt.plot(term, value, 'o', xi, yi,'-')#
#plt.legend(['data', 'linear'], loc='best')
#plt.show()

In [12]:
#xi,yi = curvefitting(np.array([0.25,0.3,0.5,1.1,1.2,1.9,2.15,2.5,3])/100,inter="linear",extro="tar")
#import matplotlib.pyplot as plt
#term = np.array([0.25,1,2,3,5,7,10,20,30])
#value = np.array([0.25,0.3,0.5,1.1,1.2,1.9,2.15,2.5,3])/100
#plt.plot(term, value, 'o', xi, yi, '-')
#plt.legend(['data', 'linear'], loc='best')
#plt.show()

In [13]:
import numpy as np
def bondPrice(terms,rates,term,freq,coupon,redemption,inter="linear",extro="tar",expectedRate=0.04):
	xi,yi = curvefitting(rates,terms,inter=inter,extro=extro,target=expectedRate)
	f = interp1d(xi,yi)
	if term > 0.25:
		bondvalue = redemption*(1+coupon*freq)/(1+f(term))**term
	else:
		bondvalue = redemption
	for i in np.arange(term-freq,0.01,-freq):
		bondvalue = bondvalue + redemption*coupon*freq/(1+f(i))**(i)
	return bondvalue
term = np.array([0.25,1,2,3,5,7,10,20,30])
value = np.array([0.575593169+0.457631,0.842160583+0.457631,1.20,1.47,1.93,2.25,2.45,2.79,3.06])/100
bondPrice(term,value,0.75,1,1.14/100,21705)

C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  
C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  from ipykernel import kernelapp as app


21755.153866790886

In [14]:
#thmf=sampleMF(var,fundmap,hmf,har,var1chol,normalChol,recessionChol,10,30,stochastic=True)

In [15]:
def bondReturn(thmf,mix,migrationM,dyn=10,n=10,rating=2,rebalance=0.5,bondfreq=0.25,inter="linear",extro="tar",expectedRate=0.04):
#rating: 0-Govt Bond; 1-AAA; 2-AA; 3-A; 4-BBB
	term = np.array([0.25,1,2,3,5,7,10,20,30])
	initialVal = 1000000
	migrationM = np.array(migrationM)/100
	recoveryM = migrationM[:,6]*100
	mix = np.array(mix)
	#rating=2
	#bondfreq=0.25
	MVM = mix[rating] * initialVal
	value = thmf[0,[4,9,10,11,12,13,14,15,16]]
	if rating>0:
		value = value + thmf[0,16+rating*2]
	default= np.append(0,thmf[0,[19,21,23,25]])/100
	xnew = np.linspace(0.25,30,num=120,endpoint=True)
	inter="linear"
	if inter=="linear":
		f = interp1d(term,value)
	else:
		f = interp1d(term,value,kind='cubic')
	CouponM = np.array(f(xnew))
	FVM = np.copy(MVM)
	for i in range(0,120):
		FVM[i] = FVM[i]/bondPrice(term,value/100,(i+1)*0.25,bondfreq,CouponM[i]/100,1,inter=inter,extro=extro,expectedRate=expectedRate)
	nBSM = np.repeat(0.0,120)
	RedemptionM = np.repeat(0.0,120)
	tMVM = np.copy(MVM)
	tFVM = np.copy(FVM)
	tBSM = np.copy(nBSM)
	tRDM = np.copy(RedemptionM)
	tCRM = np.copy(CouponM)
	cashRtn = []
	priceRtn = []

	for i in range(1,thmf.shape[0]):
		bcurve = thmf[i,[4,9,10,11,12,13,14,15,16]]
		bcurve = np.vstack([bcurve,bcurve + thmf[i,18]])
		bcurve = np.vstack([bcurve,bcurve[0] + thmf[i,20]])
		bcurve = np.vstack([bcurve,bcurve[0] + thmf[i,22]])
		bcurve = np.vstack([bcurve,bcurve[0] + thmf[i,24]])
		bcurve = np.vstack([bcurve,bcurve[4] + 2.499197])
		#if i==1:
			#print bcurve
		nMVM = np.repeat(0.0,120)
		nFVM = np.repeat(0.0,120)
		nBSM = np.repeat(0.0,120)
		RedemptionM = np.repeat(0.0,120)

		totalBS = 0.0
		xnew = np.linspace(0.25,30,num=120,endpoint=True)
		inter="linear"
		if inter=="linear":
			f = interp1d(term,bcurve[rating])
		else:
			f = interp1d(term,bcurve[rating],kind='cubic')
		nCouponM = np.array(f(xnew))
		for j in range(0,120):
			for k in range(0,6):
				if k==rating:
					nMVM[j] = bondPrice(term,bcurve[rating]/100,(j+1)/4.0-0.25,bondfreq,CouponM[j]/100,FVM[j],inter=inter,extro=extro,expectedRate=expectedRate)*migrationM[rating,rating]*(1-default[rating]*(1-recoveryM[rating]))
					nFVM[j] = FVM[j]*migrationM[rating,rating]*(1-default[rating]*(1-recoveryM[rating]))
				else:
					totalBS = totalBS + bondPrice(term,bcurve[k]/100,(j+1)/4.0-0.25,bondfreq,CouponM[j]/100,FVM[j],inter=inter,extro=extro,expectedRate=expectedRate)*migrationM[rating,k]*(1-default[rating]*(1-recoveryM[rating]))
			if j==0:
				RedemptionM[j] = nMVM[j]*(1-default[rating]*(1-recoveryM[rating]))
			if ((j+1)%int(bondfreq*4)==1 and bondfreq>0.25) or bondfreq==0.25:
				RedemptionM[j] = RedemptionM[j] + FVM[j]*CouponM[j]/100*bondfreq*(1-default[rating]*(1-recoveryM[rating]))
	
		totalBS = totalBS + np.sum(RedemptionM)
		if rebalance==0:
			nBSM=totalBS*mix[rating]
			nBSFM = np.repeat(0.0,120)
			for j in range(0,120):
				nBSFM[j] = nBSM[j]/bondPrice(term,bcurve[rating]/100,(j+1)/4.0,bondfreq,nCouponM[j]/100,1)
			for j in range(1,120):
				MVM[j-1]=nMVM[j]
				FVM[j-1]=nFVM[j]
			MVM[119]=0
			FVM[119]=0
			for j in range(1,120):
				if (FVM[j-1]+nBSFM[j-1]) != 0:
					CouponM[j-1]=(FVM[j-1]*CouponM[j]+nBSFM[j-1]*nCouponM[j-1])/(FVM[j-1]+nBSFM[j-1])
				else:
					CouponM[j-1]=CouponM[j]
			CouponM[119]=nCouponM[119]
			MVM = MVM + nBSM
			FVM = FVM + nBSFM
		elif i%int(rebalance*4)==0:
			totalBS = totalBS + np.sum(nMVM)
			nBSM=totalBS*mix[rating]
			for j in range(0,119):
				nBSM[j] = nBSM[j]-nMVM[j+1]
			nBSFM = np.repeat(0.0,120)
			for j in range(0,120):
				if nBSM[j]<0:
					nBSFM[j] = nBSM[j]/bondPrice(term,bcurve[rating]/100,(j+1)/4.0,bondfreq,CouponM[j+1]/100,1,inter=inter,extro=extro,expectedRate=expectedRate)
				else:
					nBSFM[j] = nBSM[j]/bondPrice(term,bcurve[rating]/100,(j+1)/4.0,bondfreq,nCouponM[j]/100,1,inter=inter,extro=extro,expectedRate=expectedRate)				
			for j in range(1,120):
				MVM[j-1]=nMVM[j]
				FVM[j-1]=nFVM[j]
			MVM[119]=0
			FVM[119]=0
			for j in range(1,120):
				if nBSM[j-1]>0 and (FVM[j-1]+nBSFM[j-1])!=0:
					CouponM[j-1]=(FVM[j-1]*CouponM[j]+nBSFM[j-1]*nCouponM[j-1])/(FVM[j-1]+nBSFM[j-1])
				else:
					CouponM[j-1]=CouponM[j]
			CouponM[119]=nCouponM[119]
			MVM = MVM + nBSM
			FVM = FVM + nBSFM
		else:
			nBSM=totalBS*mix[rating]
			nBSFM = np.repeat(0.0,120)
			for j in range(0,120):
				nBSFM[j] = nBSM[j]/bondPrice(term,value/100,(j+1)/4,bondfreq,nCouponM[j]/100,1,inter=inter,extro=extro,expectedRate=expectedRate)
			for j in range(1,120):
				MVM[j-1]=nMVM[j]
				FVM[j-1]=nFVM[j]
			MVM[119]=0
			FVM[119]=0
			for j in range(1,120):
				if (FVM[j-1]+nBSFM[j-1]) != 0:
					CouponM[j-1]=(FVM[j-1]*CouponM[j]+nBSFM[j-1]*nCouponM[j-1])/(FVM[j-1]+nBSFM[j-1])
				else:
					CouponM[j-1]=CouponM[j]
			CouponM[119]=nCouponM[119]
			MVM = MVM + nBSM
			FVM = FVM + nBSFM
		tMVM = np.vstack([tMVM,MVM])
		tFVM = np.vstack([tFVM,FVM])
		tBSM = np.vstack([tBSM,nBSM])
		tRDM = np.vstack([tRDM,RedemptionM])
		tCRM = np.vstack([tCRM,CouponM])
		cashRtn = np.append(cashRtn,np.sum(RedemptionM)/initialVal)
		priceRtn = np.append(priceRtn,(np.sum(MVM)-np.sum(RedemptionM))/initialVal-1)
		default= np.append(0,thmf[i,[19,21,23,25]])/100
		initialVal = np.sum(MVM)
	#print tMVM[range(0,5),:]
	#print tFVM[range(0,5),:]
	#print tBSM[range(0,5),:]
	#print tRDM[range(0,5),:]
	#print tCRM[range(0,5),:]
	return cashRtn,priceRtn


migrationM = pd.read_csv(migration)
mix=pd.read_csv(termmix)
#cashR,priceR = bondReturn(thmf,mix,migrationM,dyn=10,n=10,rating=0,rebalance=0,bondfreq=0.25,inter="linear",extro="tar",expectedRate=0.04)
#cashR,priceR = bondReturn(check,mix,migrationM,dyn=60,n=60,rating=4,rebalance=1,bondfreq=1,inter="linear",extro="cfr",expectedRate=0.04)

In [16]:
def AddBondReturn(thmf,mix,migrationM,dyn=10,n=10,rebalance=0,bondfreq=0.25,inter="linear",extro="tar",expectedRate=0.04):
	for ir in range(0,5):
		cashR,priceR = bondReturn(thmf=thmf,mix=mix,migrationM=migrationM,dyn=10,n=n,rating=ir,rebalance=rebalance,bondfreq=bondfreq,inter=inter,extro=extro,expectedRate=expectedRate)
		thmf = np.append(thmf,np.transpose(np.vstack([np.insert(cashR,0,0),np.insert(priceR,0,0)])),axis=1)
		#print "rating ", ir, " done"
	return thmf

In [17]:
def AdjustBondReturn(thmf,thmfBase,thmfFull,dyn=10,n=10):
	thmf = np.append(thmf,thmfBase[:,range(42,52)],axis=1)
	for i in range(0,n):
		if i> dyn:
			for ir in [43,45,47,49,51]:
				thmf[i,ir] = thmfBase[i,ir]+(thmfBase[i,14]-thmfFull[i,14])/400
			for ir in [45,47,49,51]:
				thmf[i,ir] = thmfBase[i,ir]+(thmfFull[i,ir-27]-thmfBase[i,ir-27] - thmfFull[i,ir-26]+thmfBase[i,ir-26])/400
	return thmf

In [18]:
#thmf = AddBondReturn(thmf,mix,migrationM,dyn=10,n=10,rebalance=0,bondfreq=0.25,inter="linear",extro="tar",expectedRate=0.04)
#check = AddBondReturn(check,mix,migrationM,dyn=60,n=60,rebalance=1,bondfreq=1,inter="linear",extro="tar",expectedRate=0.04)

In [19]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, date, time

fMI = 0.01
mMI = 0.01
salaryGr = 0.02
salaryAvgPeriod = 5
benefitRate = 0.01
COLA = 0.8
maxCOLA = 0.05
lumpSumProb = 0.1
lumpSumDR = 0.04
valuationDate = date(2016,12,31)
planliab = 10000000.
#valuationDate.year
dateOfBirth = date(1981,6,30)
startDate = date(2005,1,31)
salary = 80000.
retireDate = date(2045,6,30)
weight = 0.08
occupation = 4
salaryMultiple = np.array([0.9,1.0,1.1,1.2,1.5])
gender = "F"
dyn=10
inter="linear"
extro="tar"
target=0.04
saa=[0.,0.,0.5,0.,0.,0.5,0.,0.,0.,0.,0.]

In [20]:
def pbo(gender,dateOfBirth, startDate,retireDate,salary,occupation,weight,thmf,dyn,inter="linear",extro="tar",target=0.04):
	#saa = np.array(saa)
	wTerms = int(max(0,(retireDate.year - valuationDate.year)*4+(retireDate.month-valuationDate.month)/4))+1
	retireArray = np.repeat(0,wTerms)
	retireArray = np.append(retireArray,np.repeat(1,361-wTerms))
	servicePeriod = (valuationDate.year - startDate.year) + (valuationDate.month - startDate.month)/12.
	spArray = np.repeat(servicePeriod,361)
	age = (valuationDate.year - dateOfBirth.year) + (valuationDate.month - startDate.month)/12.
	ageArray = np.repeat(age,361)
	qScn = thmf.shape[0]-1
	wageIndex = np.repeat(1.,361)
	COLAArray = np.repeat(1.,361)
	salaryArray = np.repeat(salary,361)
	supSalaryArray = np.repeat(salary,361+salaryAvgPeriod*4-1)
	for i in range(0,salaryAvgPeriod*4-1):
		if i%4 == 0:
			supSalaryArray[i] = salary/(1+salaryGr*salaryMultiple[occupation-1])**(salaryAvgPeriod-1-i/4)
		else:
			supSalaryArray[i] = supSalaryArray[i-1]
	avgSalaryArray = np.repeat(np.average(supSalaryArray[0:salaryAvgPeriod*4]),361)
	accBenefitArray = np.repeat(0.,361)
	prjBenefitArray = np.repeat(0.,361)
	paymentArray = np.repeat(0.,361)
	prjPaymentArray = np.repeat(0.,361)

	mortTables = pd.read_csv(Mortality)
	mortTables = np.array(mortTables)
	if gender == "M":
		baseMortArray = np.repeat(1-(1-mortTables[min(110,int(round(age))),1])**0.25,361)
	else:
		baseMortArray = np.repeat(1-(1-mortTables[min(110,int(round(age))),2])**0.25,361)

	survivorship = np.repeat(1.,361)

	ldrArray = np.repeat(1.,361)
	pcdrArray = np.repeat(1.,361)
	lsdrArray = np.repeat(1.,361)

	value = thmf[dyn,[4,9,10,11,12,13,14,15,16]]
	value = value + thmf[dyn,20]
	xi,yi = curvefitting(value/100,term=np.array([0.25,1,2,3,5,7,10,20,30]),inter=inter,extro=extro,target=target)

	for t in range(1,361):

		if retireArray[t] == 1:
			spArray[t] = spArray[t-1]
		else:
			spArray[t] = servicePeriod + t/4.

		ageArray[t] = age + t/4.

		if retireArray[t] == 1:
			wageIndex[t] = wageIndex[t-1]
		elif t%4 == 0:
			wageIndex[t] = wageIndex[t-1]*(1+((1+thmf[min(t,qScn),46]/100)*(1+thmf[min(t-1,qScn),46]/100)*(1+thmf[min(t-2,qScn),46]/100)*(1+thmf[min(t-3,qScn),46]/100)-1)*salaryMultiple[occupation-1])
		else:
			wageIndex[t] = wageIndex[t-1]

		if retireArray[t] == 0:
			COLAArray[t] = 1
		elif t%4 == 0:
			COLAArray[t] = COLAArray[t-1] * min(1+maxCOLA,(1-COLA)+COLA*(1+thmf[min(t,qScn),17]/100)*(1+thmf[min(t-1,qScn),17]/100)*(1+thmf[min(t-2,qScn),17]/100)*(1+thmf[min(t-3,qScn),17]/100))
		else:
			COLAArray[t] = COLAArray[t-1]

		if retireArray[t] == 0:
			salaryArray[t] = salary * wageIndex[t]
			supSalaryArray[t+salaryAvgPeriod*4-1] = salaryArray[t]
		else:
			salaryArray[t] = salaryArray[t-1]
			supSalaryArray[t+salaryAvgPeriod*4-1] = salaryArray[t]

		avgSalaryArray[t] = np.average(supSalaryArray[t:(t+salaryAvgPeriod*4)])

		accBenefitArray[t] = avgSalaryArray[t]*benefitRate*min(servicePeriod+dyn/4.,spArray[t])*COLAArray[t]/4
		prjBenefitArray[t] = avgSalaryArray[t]*benefitRate*spArray[t]*COLAArray[t]/4

		if gender == "M":
			baseMortArray[t] = 1-(1-mortTables[min(110,int(round(ageArray[t]))),1])**0.25
			survivorship[t] = survivorship[t-1]*(1-baseMortArray[t]*(1-mMI))
		else:
			baseMortArray[t] = 1-(1-mortTables[min(110,int(round(ageArray[t]))),2])**0.25
			survivorship[t] = survivorship[t-1]*(1-baseMortArray[t]*(1-fMI))

		
		lsdrArray[t] = lsdrArray[t-1]/(1+lumpSumDR)**0.25
		mt = thmf.shape[0]-1
		if t <dyn:
			ldrArray[t] = 1
			pcdrArray[t] = 1
		else:
			ldrArray[t] = ldrArray[t-1]/(1+yi[t-dyn])**0.25
			rtns = np.array([(thmf[min(mt,t),42]+thmf[min(mt,t),43])*100,(thmf[min(mt,t),44]+thmf[min(mt,t),45])*100,(thmf[min(mt,t),46]+thmf[min(mt,t),47])*100,(thmf[min(mt,t),48]+thmf[min(mt,t),49])*100,(thmf[min(mt,t),50]+thmf[min(mt,t),51])*100,thmf[min(mt,t),26]/4.+thmf[min(mt,t),27],thmf[min(mt,t),28]/4.+thmf[min(mt,t),29],thmf[min(mt,t),30]/4.+thmf[min(mt,t),31],thmf[min(mt,t),32]/4.+thmf[min(mt,t),33],thmf[min(mt,t),36]+thmf[min(mt,t),37],thmf[min(mt,t),38]+thmf[min(mt,t),39]])
			pcdrArray[t] = pcdrArray[t-1]/(1+np.sum(np.multiply(saa,rtns))/100)
	
	lumpSum = np.sum(np.multiply(np.multiply(np.multiply(lsdrArray,accBenefitArray),survivorship),retireArray))/lsdrArray[wTerms]/survivorship[wTerms]
	
	for t in range(1,361):
		if wTerms == 0:
			paymentArray[t] = accBenefitArray[t]*retireArray[t]*survivorship[t]
			prjPaymentArray[t] = prjBenefitArray[t]*retireArray[t]*survivorship[t]
		elif t == wTerms:
			paymentArray[t] = accBenefitArray[t]*retireArray[t]*survivorship[t]*(1-lumpSumProb)+lumpSum*lumpSumProb
			prjPaymentArray[t] = prjBenefitArray[t]*retireArray[t]*survivorship[t]
		else:
			paymentArray[t] = accBenefitArray[t]*retireArray[t]*survivorship[t]*(1-lumpSumProb)
			prjPaymentArray[t] = prjBenefitArray[t]*retireArray[t]*survivorship[t]
	
	dynArray = np.repeat(0,361)
	for t in range(0,361):
		if t > dyn:
			dynArray[t]=1

	dynPBO = np.sum(np.multiply(np.multiply(paymentArray,ldrArray),dynArray))/survivorship[dyn]
	if retireArray[dyn]==0:
		planCost = 0.25/max(spArray)*np.sum(np.multiply(prjPaymentArray,pcdrArray))/survivorship[dyn]
	else:
		planCost = 0
	return [dynPBO,planCost,paymentArray[dyn],prjPaymentArray[dyn]]

#pbo(gender,dateOfBirth, startDate,retireDate,salary,occupation,weight,check,60,saa,inter,extro,target)

In [21]:
employees = pd.read_csv(Census)

In [22]:
def liabMultiple(employees,thmf,planliab=10000000.,inter="linear",extro="tar",target=0.04):
	liabSum = 0
	employees = np.array(employees)
	for employID in range(0,employees.shape[0]):
		gender=employees[employID,2]
		dateOfBirth=date(*map(int, employees[employID,3].split('-')))
		startDate=date(*map(int, employees[employID,4].split('-')))
		retireDate=date(*map(int, employees[employID,6].split('-')))
		salary=employees[employID,5]
		occupation=employees[employID,7]
		weight=employees[employID,8]
		wTerms = int(max(0,(retireDate.year - valuationDate.year)*4+(retireDate.month-valuationDate.month)/4))+1
		retireArray = np.repeat(0,wTerms)
		retireArray = np.append(retireArray,np.repeat(1,361-wTerms))
		servicePeriod = (valuationDate.year - startDate.year) + (valuationDate.month - startDate.month)/12.
		spArray = np.repeat(servicePeriod,361)
		age = (valuationDate.year - dateOfBirth.year) + (valuationDate.month - startDate.month)/12.
		ageArray = np.repeat(age,361)
		qScn = thmf.shape[0]-1
		wageIndex = np.repeat(1.,361)
		COLAArray = np.repeat(1.,361)
		salaryArray = np.repeat(salary,361)
		supSalaryArray = np.repeat(salary,361+salaryAvgPeriod*4-1)
		for i in range(0,salaryAvgPeriod*4-1):
			if i%4 == 0:
				supSalaryArray[i] = salary/(1+salaryGr*salaryMultiple[occupation-1])**(salaryAvgPeriod-1-i/4)
			else:
				supSalaryArray[i] = supSalaryArray[i-1]
		avgSalaryArray = np.repeat(np.average(supSalaryArray[0:salaryAvgPeriod*4]),361)
		accBenefitArray = np.repeat(0.,361)
		paymentArray = np.repeat(0.,361)

		mortTables = pd.read_csv(Mortality)
		mortTables = np.array(mortTables)
		if gender == "M":
			baseMortArray = np.repeat(1-(1-mortTables[min(110,int(round(age))),1])**0.25,361)
		else:
			baseMortArray = np.repeat(1-(1-mortTables[min(110,int(round(age))),2])**0.25,361)

		survivorship = np.repeat(1.,361)

		ldrArray = np.repeat(1.,361)
		lsdrArray = np.repeat(1.,361)

		value = thmf[0,[4,9,10,11,12,13,14,15,16]]
		value = value + thmf[0,20]
		xi,yi = curvefitting(value/100,term=np.array([0.25,1,2,3,5,7,10,20,30]),inter=inter,extro=extro,target=target)

		for t in range(1,361):

			if retireArray[t] == 1:
				spArray[t] = spArray[t-1]
			else:
				spArray[t] = servicePeriod + t/4.

			ageArray[t] = age + t/4.

			if retireArray[t] == 1:
				wageIndex[t] = wageIndex[t-1]
			elif t%4 == 0:
				wageIndex[t] = wageIndex[t-1]*(1+((1+thmf[min(t,qScn),38]/100)*(1+thmf[min(t-1,qScn),38]/100)*(1+thmf[min(t-2,qScn),38]/100)*(1+thmf[min(t-3,qScn),38]/100)-1)*salaryMultiple[occupation-1])
			else:
				wageIndex[t] = wageIndex[t-1]

			if retireArray[t] == 0:
				COLAArray[t] = 1
			elif t%4 == 0:
				COLAArray[t] = COLAArray[t-1] * min(1+maxCOLA,(1-COLA)+COLA*(1+thmf[min(t,qScn),17]/100)*(1+thmf[min(t-1,qScn),17]/100)*(1+thmf[min(t-2,qScn),17]/100)*(1+thmf[min(t-3,qScn),17]/100))
			else:
				COLAArray[t] = COLAArray[t-1]

			if retireArray[t] == 0:
				salaryArray[t] = salary * wageIndex[t]
				supSalaryArray[t+salaryAvgPeriod*4-1] = salaryArray[t]
			else:
				salaryArray[t] = salaryArray[t-1]
				supSalaryArray[t+salaryAvgPeriod*4-1] = salaryArray[t]

			avgSalaryArray[t] = np.average(supSalaryArray[t:(t+salaryAvgPeriod*4)])

			accBenefitArray[t] = avgSalaryArray[t]*benefitRate*servicePeriod*COLAArray[t]/4

			if gender == "M":
				baseMortArray[t] = 1-(1-mortTables[min(110,int(round(ageArray[t]))),1])**0.25
				survivorship[t] = survivorship[t-1]*(1-baseMortArray[t]*(1-mMI))
			else:
				baseMortArray[t] = 1-(1-mortTables[min(110,int(round(ageArray[t]))),2])**0.25
				survivorship[t] = survivorship[t-1]*(1-baseMortArray[t]*(1-fMI))

		
			lsdrArray[t] = lsdrArray[t-1]/(1+lumpSumDR)**0.25
			ldrArray[t] = ldrArray[t-1]/(1+yi[t])**0.25
	
		lumpSum = np.sum(np.multiply(np.multiply(np.multiply(lsdrArray,accBenefitArray),survivorship),retireArray))/lsdrArray[wTerms]/survivorship[wTerms]
		for t in range(1,361):
			if wTerms == 0:
				paymentArray[t] = accBenefitArray[t]*retireArray[t]*survivorship[t]
			elif t == wTerms:
				paymentArray[t] = accBenefitArray[t]*retireArray[t]*survivorship[t]*(1-lumpSumProb)+lumpSum*lumpSumProb
			else:
				paymentArray[t] = accBenefitArray[t]*retireArray[t]*survivorship[t]*(1-lumpSumProb)

		dynPBO = np.sum(np.multiply(paymentArray,ldrArray))
		#if employID==8:
			#print accBenefitArray,paymentArray
		liabSum = liabSum + dynPBO*weight
	multiple = planliab/liabSum
	return multiple
	
multiple = liabMultiple(employees,check,planliab,inter,extro,target)

C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  
C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  from ipykernel import kernelapp as app


In [23]:
multiple

50.230445964671105

In [24]:
def aggPBO(employees,thmf,dyn,multiple,planliab=10000000.,inter="linear",extro="tar",target=0.04):
	aggResult = np.array([0,0,0,0])	
	employees = np.array(employees)
	#multiple = liabMultiple(employees,thmf,planliab,inter,extro,target)
#	print multiple
	for employID in range(0,employees.shape[0]):
		gender=employees[employID,2]
		dateOfBirth=date(*map(int, employees[employID,3].split('-')))
		startDate=date(*map(int, employees[employID,4].split('-')))
		retireDate=date(*map(int, employees[employID,6].split('-')))
		salary=employees[employID,5]
		occupation=employees[employID,7]
		weight=employees[employID,8]
#		print weight
		aggResult = aggResult + np.multiply(weight, np.array(pbo(gender,dateOfBirth, startDate,retireDate,salary,occupation,weight,thmf,dyn,inter,extro,target)))
	aggResult = np.multiply(aggResult, multiple)
	return aggResult
#check = tmpArrays[205:246,:]
#aggPBO(employees,check,24,saa,multiple,planliab,inter,extro,target)

In [25]:
nsim = 1
ndyn = 60
planasset = 10000000.

rebalance = 0.25
bondfreq = 0.25
expectedRate = 0.04
bs = True
possaa = saa
negsaa = saa

In [26]:
def fundproj(employees,nsim,ndyn,saa, negsaa, possaa, var,fundmap,histMF,histAR,cholMF,\
			cholNormal,cholRecession,mix,migrationM,stochastic=True,planliab=10000000.,planasset=10000000.,bs = True,\
			inter="linear",extro="tar",target=0.04,rebalance=0.5,bondfreq=0.25):
	thmf=sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,0,1,stochastic=False)
	thmfBase=sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,ndyn,ndyn,stochastic=False)
	thmfBase=AddBondReturn(thmfBase,mix,migrationM,ndyn,ndyn,rebalance,bondfreq,inter,extro,expectedRate)
	multiple = liabMultiple(employees,thmf,planliab,inter,extro,target)
	liabArray = np.array([1,0,planliab,0,0,0])
	liabArray = np.append(liabArray,np.multiply(saa,planasset))
	liabArray = np.append(liabArray,np.array([planasset,planasset/planliab,planasset-planliab]))
	tmpArray = np.copy(liabArray)
	
	for isim in range(0,nsim):
		assetArray = np.multiply(saa,planasset)
		csaa = saa
		print("Starting Simulation ", isim+1, ": ", str(datetime.now()))
		dynThmf=sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,ndyn,ndyn,stochastic)
		thmfFull=AddBondReturn(dynThmf,mix,migrationM,ndyn,ndyn,rebalance,bondfreq,inter,extro,expectedRate)
		if isim>0:
			liabArray = np.vstack([liabArray,tmpArray])
		for iq in range(0,ndyn):
			thmf = sampleMF2(dynThmf, var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,iq,ndyn)
			thmf = AdjustBondReturn(thmf,thmfBase,thmfFull,iq,ndyn)
			pboArray = aggPBO(employees,thmf,dyn,multiple,planliab,inter,extro,target)
			liabNCF = pboArray[1] - pboArray[2]
			cashRtns = np.array([thmf[iq+1,42]*100,thmf[iq+1,44]*100,thmf[iq+1,46]*100,thmf[iq+1,48]*100,thmf[iq+1,50]*100,thmf[iq+1,26]/4.,thmf[iq+1,28]/4.,thmf[iq+1,30]/4.,thmf[iq+1,32]/4.,thmf[iq+1,36],thmf[iq+1,38]])
			priceRtns = np.array([thmf[iq+1,43]*100,thmf[iq+1,45]*100,thmf[iq+1,47]*100,thmf[iq+1,49]*100,thmf[iq+1,51]*100,thmf[iq+1,27],thmf[iq+1,29],thmf[iq+1,31],thmf[iq+1,33],thmf[iq+1,37],thmf[iq+1,39]])
			cashCF = np.sum(np.multiply(assetArray,cashRtns/100))
			assetArray = assetArray+np.multiply(assetArray,priceRtns/100)
			newAssetValue = np.sum(assetArray)
			if bs == False and (cashCF+liabNCF) > 0:
				bsArray = np.multiply((cashCF+liabNCF),possaa)
			elif bs == False and (cashCF + liabNCF) <= 0:
				bsArray = np.multiply((cashCF+liabNCF),negsaa)
			else:
				bsArray = np.multiply((cashCF+liabNCF+newAssetValue),csaa)-assetArray
			assetArray = assetArray + bsArray
			pboArray = np.insert(pboArray,0,[isim+1,iq+1])
			pboArray = np.append(pboArray,assetArray)
			pboArray = np.append(pboArray,np.array([np.sum(assetArray),np.sum(assetArray)/pboArray[2],np.sum(assetArray)-pboArray[2]]))
			liabArray = np.vstack([liabArray,pboArray])
			csaa = saa#will be dynamic
			#print("Dynamic projection for Quarter ", iq+1, ": ", str(datetime.now()))
		print("Finishing Simulation ", isim+1, ": ", str(datetime.now()))
	return liabArray
#liabArrays = fundproj(employees,10,ndyn,saa, negsaa, possaa, var,fundmap,hmf,har,var1chol,normalChol,recessionChol, \
#			mix,migrationM,stochastic=True,planliab=10000000.,planasset=10000000.,bs = True, \
#			inter="linear",extro="tar",target=0.04,rebalance=0.5,bondfreq=0.25)
#np.savetxt("test.csv", liabArrays, delimiter=",")
#liabArrays

In [27]:
def createStates(nsim,ndyn,var,fundmap,histMF,histAR,cholMF,\
			cholNormal,cholRecession,mix,migrationM,stochastic=True,\
			inter="linear",extro="tar",target=0.04,rebalance=0.5,bondfreq=0.25):
	thmf=sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,0,1,stochastic=False)
	thmfBase=sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,ndyn,ndyn,stochastic=False)
	thmfBase=AddBondReturn(thmfBase,mix,migrationM,ndyn,ndyn,rebalance,bondfreq,inter,extro,expectedRate)
	tmpArray = np.copy(thmfBase)
	
	for isim in range(0,nsim):
		print("Starting Simulation ", isim+1, ": ", str(datetime.now()))
		dynThmf=sampleMF(var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,ndyn,ndyn,stochastic)
		thmfFull=AddBondReturn(dynThmf,mix,migrationM,ndyn,ndyn,rebalance,bondfreq,inter,extro,expectedRate)
		tmpArray = np.vstack([tmpArray,thmfFull])
		print("Finishing Simulation ", isim+1, ": ", str(datetime.now()))
	return tmpArray
#tmpArrays = createStates(100,ndyn,var,fundmap,hmf,har,var1chol,normalChol,recessionChol, \
#			mix,migrationM,stochastic=True, \
#			inter="linear",extro="tar",target=0.04,rebalance=0.25,bondfreq=0.25)
#np.savetxt("test100.csv", tmpArrays, delimiter=",")

In [28]:
tmpArrays=np.array(pd.read_csv("C:/dsge/py/input/test.csv",header=None))
tmpArrays.shape

(6161, 52)

In [29]:
def getNewState(employees,var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,\
			liabArray,tmpArray,sim,dyn):
	thmfBase=tmpArray[0:ndyn+1,]
	dynThmf=tmpArray[(sim*(ndyn+1)):(sim*(ndyn+1)+ndyn+1),0:42]
	thmfFull=tmpArray[(sim*(ndyn+1)):(sim*(ndyn+1)+ndyn+1),:]
	iq = dyn
	thmf = sampleMF2(dynThmf, var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,iq,ndyn)
	thmf = AdjustBondReturn(thmf,thmfBase,thmfFull,iq,ndyn)
	newState = np.concatenate((thmf[dyn,:].flatten(), thmf[max(0,dyn-1),:].flatten(), thmf[max(dyn-2,0),:].flatten()))#economic environment
	newState = np.append(newState,[np.sum(liabArray[6:17])/liabArray[2],np.sum(liabArray[6:17])-liabArray[2]])#funding ratio & surplus
	newState = np.append(newState,[liabArray[8]/np.sum(liabArray[6:17]),liabArray[11]/np.sum(liabArray[6:17]),dyn])#let's add time
	return newState

liabArray = np.array([1,0,planliab,0,0,0])
liabArray = np.append(liabArray,np.multiply(saa,planasset))
liabArray = np.append(liabArray,np.array([planasset,planasset/planliab,planasset-planliab]))
state = getNewState(employees,var,fundmap,hmf,har,var1chol,normalChol,recessionChol,\
			liabArray,tmpArrays,5,0)
state.shape

(161,)

In [30]:
def getLSTMState(employees,var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,\
			liabArray,tmpArray,sim,dyn):
	thmfBase=tmpArray[0:ndyn+1,]
	dynThmf=tmpArray[(sim*(ndyn+1)):(sim*(ndyn+1)+ndyn+1),0:42]
	thmfFull=tmpArray[(sim*(ndyn+1)):(sim*(ndyn+1)+ndyn+1),:]
	iq = dyn
	thmf = sampleMF2(dynThmf, var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,iq,ndyn)
	thmf = AdjustBondReturn(thmf,thmfBase,thmfFull,iq,ndyn)
	newState = thmf[dyn,:].flatten()
	newState = np.append(newState,[np.sum(liabArray[6:17])/liabArray[2],np.sum(liabArray[6:17])-liabArray[2]])
	newState = np.append(newState,[liabArray[8]/np.sum(liabArray[6:17]),liabArray[11]/np.sum(liabArray[6:17])])
	return newState

liabArray = np.array([1,0,planliab,0,0,0])
liabArray = np.append(liabArray,np.multiply(saa,planasset))
liabArray = np.append(liabArray,np.array([planasset,planasset/planliab,planasset-planliab]))
state = getLSTMState(employees,var,fundmap,hmf,har,var1chol,normalChol,recessionChol,\
			liabArray,tmpArrays,5,0)
state.shape

(56,)

In [103]:
def getReward(employees, saa, action,var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,\
			liabArray,tmpArray,sim,dyn,multiple,planliab=10000000.,bs = True,\
			inter="linear",extro="tar",target=0.04):
	thmfBase=tmpArray[0:(ndyn+1),]
	
	assetArray = liabArray[6:17]
	if action==0:
		csaa = saa
	elif action == 1:
		csaa = np.minimum(1,np.maximum(0,saa + np.array([0.,0.,0.03,0.,0.,-0.03,0.,0.,0.,0,0])))
	else:
		csaa = np.minimum(1,np.maximum(0,saa + np.array([0.,0.,-0.03,0.,0.,0.03,0.,0.,0.,0,0])))

	dynThmf=tmpArray[(sim*(ndyn+1)):(sim*(ndyn+1)+ndyn+1),0:42]
	thmfFull=tmpArray[(sim*(ndyn+1)):(sim*(ndyn+1)+ndyn+1),:]
	iq = dyn
	thmf = sampleMF2(dynThmf, var,fundmap,histMF,histAR,cholMF,cholNormal,cholRecession,iq,ndyn)
	thmf = AdjustBondReturn(thmf,thmfBase,thmfFull,iq,ndyn)
	pboArray = aggPBO(employees,thmf,dyn,multiple,planliab,inter,extro,target)
	liabNCF = pboArray[1] - pboArray[2]
	cashRtns = np.array([thmf[iq+1,42]*100,thmf[iq+1,44]*100,thmf[iq+1,46]*100,thmf[iq+1,48]*100,thmf[iq+1,50]*100,thmf[iq+1,26]/4.,thmf[iq+1,28]/4.,thmf[iq+1,30]/4.,thmf[iq+1,32]/4.,thmf[iq+1,36],thmf[iq+1,38]])
	priceRtns = np.array([thmf[iq+1,43]*100,thmf[iq+1,45]*100,thmf[iq+1,47]*100,thmf[iq+1,49]*100,thmf[iq+1,51]*100,thmf[iq+1,27],thmf[iq+1,29],thmf[iq+1,31],thmf[iq+1,33],thmf[iq+1,37],thmf[iq+1,39]])
	cashCF = np.sum(np.multiply(assetArray,cashRtns/100))
	assetArray = assetArray+np.multiply(assetArray,priceRtns/100)
	newAssetValue = np.sum(assetArray)
	if bs == False and (cashCF+liabNCF) > 0:
		bsArray = np.multiply((cashCF+liabNCF),csaa)
	elif bs == False and (cashCF + liabNCF) <= 0:
		bsArray = np.multiply((cashCF+liabNCF),csaa)
	else:
		bsArray = np.multiply((cashCF+liabNCF+newAssetValue),csaa)-assetArray
	assetArray = assetArray + bsArray
	pboArray = np.insert(pboArray,0,[sim,iq])
	pboArray = np.append(pboArray,assetArray)
	pboArray = np.append(pboArray,np.array([np.sum(assetArray),np.sum(assetArray)/pboArray[2],np.sum(assetArray)-pboArray[2]]))
	#reward = (np.sum(pboArray[6:17])-pboArray[3])/np.sum(liabArray[6:17])-(pboArray[2]-pboArray[3])/liabArray[2]#reward is asset return - liability return
	reward = - np.sum(liabArray[6:17])/liabArray[2] + np.sum(pboArray[6:17])/pboArray[2]#reward is change in funding ratio
	#print np.sum(pboArray[6:17]),pboArray[3],pboArray[4],np.sum(liabArray[6:17]),pboArray[2],liabArray[2],reward
	return reward,pboArray,csaa

saa = [0,0,0.5,0,0,0.5,0,0,0,0,0]
liabArray = np.array([1,0,planliab,0,0,0])
liabArray = np.append(liabArray,np.multiply(saa,planasset))
liabArray = np.append(liabArray,np.array([planasset,planasset/planliab,planasset-planliab]))
newInput = np.copy(liabArray)
#for i in range(0,40):
#    rwd,newInput,saa = getReward(employees,saa, -1,var,fundmap,hmf,har,var1chol,normalChol,recessionChol,\
#                newInput,tmpArrays,5,i,multiple,planliab,bs = True,inter="linear",extro="tar",target=0.04)
#    print rwd,saa

In [123]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as T
from collections import namedtuple

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.lin1 = nn.Linear(161, 360)
#        self.bn1 = nn.BatchNorm1d(360)
        self.lin2 = nn.Linear(360, 360)
#        self.bn2 = nn.BatchNorm1d(360)
        self.lin3 = nn.Linear(360, 480)
#        self.bn3 = nn.BatchNorm1d(480)
        self.head = nn.Linear(480, 3)

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
#        x = F.relu(self.bn3(self.lin3(x)))
#        x = F.softmax(self.lin1(x), dim=0)
#        x = F.softmax(self.lin2(x), dim=0)
        x = F.softmax(self.lin3(x), dim=0)
        return self.head(x.view(x.size(0),-1))


In [124]:
BATCH_SIZE = 72
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
STATIC_UPDATE = 1

model = DQN()
static_model = DQN()
static_model.load_state_dict(model.state_dict())
static_model.eval()

if use_cuda:
    model.cuda()

optimizer = optim.RMSprop(model.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return model(state.type(FloatTensor)).max(1)[1].view(1, 1)
    else:
        return LongTensor([[random.randrange(3)]])


last_sync = 0

In [125]:
def optimize_model():
    global last_sync
    if len(memory) < BATCH_SIZE:
        return 0.0, 0.0
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see http://stackoverflow.com/a/19343/3343043 for
    # detailed explanation).
    batch = Transition(*zip(*transitions))
    #print something

    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)))

    # We don't want to backprop through the expected action values and volatile
    # will save us on temporarily changing the model parameters'
    # requires_grad to False!
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    #print reward_batch

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    #print model(state_batch.type(FloatTensor))
    state_action_values = model(state_batch.type(FloatTensor)).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    next_state_values = torch.zeros(BATCH_SIZE).type(Tensor)
    next_state_values[non_final_mask] = static_model(non_final_next_states.type(FloatTensor)).max(1)[0].detach()
    # Now, we don't want to mess up the loss with a volatile flag, so let's
    # clear it. After this, we'll just end up with a Variable that has
    # requires_grad=False
    #next_state_values.volatile = False
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    #print (expected_state_action_values)
    # Compute Huber loss
    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    #print(loss)#, reward_batch
    #print (torch.mean(expected_state_action_values))
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return float(loss), float(torch.mean(expected_state_action_values))

In [126]:
num_episodes = 1000
tsaa = [0,0,0.5,0,0,0.5,0,0,0,0,0]

eps = []
qs = []
lss = []
vals = []

for i_episode in range(num_episodes):
    # Initialize the environment and state
    saa = [0,0,0.5,0,0,0.5,0,0,0,0,0]
    liabArray = np.array([1,0,planliab,0,0,0])
    liabArray = np.append(liabArray,np.multiply(saa,planasset))
    liabArray = np.append(liabArray,np.array([planasset,planasset/planliab,planasset-planliab]))
    if (i_episode+1)% 100 == 0:
        isim = 100
    else:
        isim = (i_episode+1)% 100
    state = getNewState(employees,var,fundmap,hmf,har,var1chol,normalChol,recessionChol,\
                liabArray,tmpArrays,isim,0)
    newInput = np.copy(liabArray)
    state = torch.from_numpy(state).view(1,161)       
    for idyn in range(0,40):#60
        # Select and perform an action
        action = select_action(state)
        #print action
        reward,newInput,saa = getReward(employees,saa,action[0,0],var,fundmap,hmf,har,var1chol,normalChol,recessionChol,\
                newInput,tmpArrays,isim,idyn,multiple,planliab,bs = True,inter="linear",extro="tar",target=0.04)
        #print [i_episode,idyn,reward, newInput[2],newInput[3],sum(newInput[6:17])]#, saa
        tsaa = np.vstack([tsaa,np.array(saa)])
        #reward = getReward(action[0, 0])
        reward = Tensor([reward])
        #print reward
        if idyn == 59:
            next_state = None
        else:
            next_state = getNewState(employees,var,fundmap,hmf,har,var1chol,normalChol,recessionChol,\
                newInput,tmpArrays,isim,idyn+1)
            #print next_state, next_state.shape
            next_state = torch.from_numpy(next_state).view(1,161) 
        # Store the transition in memory
        memory.push(state, action, next_state, reward)
        #print state.size(), action.size(), next_state.size(),reward.size()
        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        loss, val =optimize_model()# 
        lss.append(loss)
        vals.append(val)
        eps.append(i_episode+1)
        qs.append(idyn+1)
    print("Episode: ",i_episode+1)
    if i_episode % STATIC_UPDATE == 0:
        static_model.load_state_dict(model.state_dict())
df = pd.DataFrame(list(zip(eps, qs, lss, vals)), 
            columns =['episode', 'quarter', 'loss','val'])

df.to_csv('C:/dsge/py/output/t'+str(GAMMA)+'_'+str(BATCH_SIZE)+'.csv')
print('Complete')

C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  
C:\Users\kshang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  from ipykernel import kernelapp as app


Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
Episode:  10
Episode:  11
Episode:  12
Episode:  13
Episode:  14
Episode:  15
Episode:  16
Episode:  17
Episode:  18
Episode:  19
Episode:  20
Episode:  21
Episode:  22
Episode:  23
Episode:  24
Episode:  25
Episode:  26
Episode:  27
Episode:  28
Episode:  29
Episode:  30
Episode:  31
Episode:  32
Episode:  33
Episode:  34
Episode:  35
Episode:  36
Episode:  37
Episode:  38
Episode:  39
Episode:  40
Episode:  41
Episode:  42
Episode:  43
Episode:  44
Episode:  45
Episode:  46
Episode:  47
Episode:  48
Episode:  49
Episode:  50
Episode:  51
Episode:  52
Episode:  53
Episode:  54
Episode:  55
Episode:  56
Episode:  57
Episode:  58
Episode:  59
Episode:  60
Episode:  61
Episode:  62
Episode:  63
Episode:  64
Episode:  65
Episode:  66
Episode:  67
Episode:  68
Episode:  69
Episode:  70
Episode:  71
Episode:  72
Episode:  73
Episode:  74
Episode:  75
Episode:  76
Episode:  77
Episode:

Episode:  593
Episode:  594
Episode:  595
Episode:  596
Episode:  597
Episode:  598
Episode:  599
Episode:  600
Episode:  601
Episode:  602
Episode:  603
Episode:  604
Episode:  605
Episode:  606
Episode:  607
Episode:  608
Episode:  609
Episode:  610
Episode:  611
Episode:  612
Episode:  613
Episode:  614
Episode:  615
Episode:  616
Episode:  617
Episode:  618
Episode:  619
Episode:  620
Episode:  621
Episode:  622
Episode:  623
Episode:  624
Episode:  625
Episode:  626
Episode:  627
Episode:  628
Episode:  629
Episode:  630
Episode:  631
Episode:  632
Episode:  633
Episode:  634
Episode:  635
Episode:  636
Episode:  637
Episode:  638
Episode:  639
Episode:  640
Episode:  641
Episode:  642
Episode:  643
Episode:  644
Episode:  645
Episode:  646
Episode:  647
Episode:  648
Episode:  649
Episode:  650
Episode:  651
Episode:  652
Episode:  653
Episode:  654
Episode:  655
Episode:  656
Episode:  657
Episode:  658
Episode:  659
Episode:  660
Episode:  661
Episode:  662
Episode:  663
Episod

In [127]:
tsaa.shape

(40001, 11)

In [128]:
df = pd.DataFrame(tsaa)
df.to_csv('C:/dsge/py/output/tsaa'+str(GAMMA)+'_'+str(BATCH_SIZE)+'.csv')

In [129]:
torch.save(model,'C:/dsge/py/output/c'+str(GAMMA)+'_'+str(BATCH_SIZE)+'.torch')
#model = torch.load('nn.torch')